In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
!pip -q install timm tqdm scikit-learn pandas


In [ ]:
from pathlib import Path
import json, os, shutil
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

import timm
from tqdm import tqdm

from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# speed
torch.backends.cudnn.benchmark = True


In [ ]:
PROJECT_ROOT = Path("/content/drive/MyDrive/SkinCare_AI_Component")

META_DIR = PROJECT_ROOT / "data/11_skin_type/metadata"
INDEX_CSV = META_DIR / "image_index_skin_type.csv"
LABEL_MAP_JSON = META_DIR / "label_map_skin_type.json"

assert INDEX_CSV.exists(), f"❌ Missing: {INDEX_CSV}"
assert LABEL_MAP_JSON.exists(), f"❌ Missing: {LABEL_MAP_JSON}"

df = pd.read_csv(INDEX_CSV)
with open(LABEL_MAP_JSON, "r") as f:
    label_map = json.load(f)

id_to_label = {v: k for k, v in label_map.items()}
num_classes = len(label_map)

print("✅ Rows:", len(df))
print("✅ Label map:", label_map)
print("\nSplit counts:\n", df["split"].value_counts())
print("\nPer-split per-class:\n", pd.crosstab(df["split"], df["label_name"]))


✅ Rows: 6992
✅ Label map: {'oily': 0, 'dry': 1, 'combination': 2}

Split counts:
 split
train    4893
test     1051
val      1048
Name: count, dtype: int64

Per-split per-class:
 label_name  combination   dry  oily
split                              
test                236   446   369
train              1099  2077  1717
val                 235   445   368


In [ ]:
df_train = df[df["split"] == "train"].copy()
df_val   = df[df["split"] == "val"].copy()
df_test  = df[df["split"] == "test"].copy()

assert len(df_train) > 0 and len(df_val) > 0 and len(df_test) > 0, "❌ Empty split detected."

print("Train:", len(df_train), "Val:", len(df_val), "Test:", len(df_test))


Train: 4893 Val: 1048 Test: 1051


In [ ]:
MODEL_NAME = "convnext_tiny"

IMG_SIZE = 224
BATCH_SIZE = 128   # ✅ try 128; if OOM, reduce to 64
EPOCHS = 20
LR = 3e-4
WEIGHT_DECAY = 0.01
PATIENCE = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

OUT_MODEL_DIR = PROJECT_ROOT / "models/vision"
OUT_MODEL_DIR.mkdir(parents=True, exist_ok=True)
BEST_CKPT_PATH = OUT_MODEL_DIR / "skin_type_convnext_cleaned_best.pt"

RESULTS_DIR = PROJECT_ROOT / "results/skin_type"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
METRICS_JSON = RESULTS_DIR / "convnext_cleaned_metrics.json"

print("✅ Save best model:", BEST_CKPT_PATH)
print("✅ Save metrics:", METRICS_JSON)


Device: cuda
✅ Save best model: /content/drive/MyDrive/SkinCare_AI_Component/models/vision/skin_type_convnext_cleaned_best.pt
✅ Save metrics: /content/drive/MyDrive/SkinCare_AI_Component/results/skin_type/convnext_cleaned_metrics.json


In [ ]:
CACHE_ROOT = Path("/content/skin_type_cache")
CACHE_ROOT.mkdir(parents=True, exist_ok=True)

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

# Simple preprocessing for cache: Resize -> ToTensor -> Normalize
cache_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

def resolve_path(image_path: str) -> Path:
    p = Path(image_path)
    return p if p.is_absolute() else (PROJECT_ROOT / p)

def cache_file_name(image_path: str) -> str:
    # stable cache name from relative path
    safe = image_path.replace("/", "__").replace("\\", "__")
    return safe + ".pt"

def build_cache(df_part: pd.DataFrame, split_name: str):
    split_dir = CACHE_ROOT / split_name
    split_dir.mkdir(parents=True, exist_ok=True)

    cached = 0
    missing = 0

    for _, r in tqdm(df_part.iterrows(), total=len(df_part), desc=f"Caching {split_name}"):
        img_rel = str(r["image_path"])
        y = int(r["label_id"])

        out_pt = split_dir / cache_file_name(img_rel)
        if out_pt.exists():
            cached += 1
            continue

        img_path = resolve_path(img_rel)
        if not img_path.exists():
            missing += 1
            continue

        img = Image.open(img_path).convert("RGB")
        x = cache_tfms(img)  # Tensor [3,224,224] normalized

        torch.save({"x": x, "y": y}, out_pt)
        cached += 1

    print(f"✅ {split_name}: cached={cached}, missing={missing}")

# Build cache for all splits (only first time is slow)
build_cache(df_train, "train")
build_cache(df_val, "val")
build_cache(df_test, "test")

print("✅ Cache ready at:", CACHE_ROOT)


Caching train: 100%|██████████| 4893/4893 [31:43<00:00,  2.57it/s]


✅ train: cached=4893, missing=0


Caching val: 100%|██████████| 1048/1048 [06:35<00:00,  2.65it/s]


✅ val: cached=1048, missing=0


Caching test: 100%|██████████| 1051/1051 [06:44<00:00,  2.60it/s]

✅ test: cached=1051, missing=0
✅ Cache ready at: /content/skin_type_cache


In [ ]:
class CachedTensorDS(Dataset):
    def __init__(self, df_, split_name: str):
        self.df = df_.reset_index(drop=True)
        self.split_dir = CACHE_ROOT / split_name

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        img_rel = str(r["image_path"])
        pt_path = self.split_dir / cache_file_name(img_rel)

        item = torch.load(pt_path, map_location="cpu")
        return item["x"], int(item["y"])

train_ds = CachedTensorDS(df_train, "train")
val_ds   = CachedTensorDS(df_val, "val")
test_ds  = CachedTensorDS(df_test, "test")

train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, pin_memory=True, persistent_workers=True
)
val_loader = DataLoader(
    val_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=4, pin_memory=True, persistent_workers=True
)
test_loader = DataLoader(
    test_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=4, pin_memory=True, persistent_workers=True
)

print("✅ Cached loaders ready.")
print("Train batches:", int(np.ceil(len(train_ds)/BATCH_SIZE)))


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


✅ Cached loaders ready.
Train batches: 39


In [ ]:
model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

scaler = torch.cuda.amp.GradScaler()

print("✅ Model ready:", MODEL_NAME)


✅ Model ready: convnext_tiny


/tmp/ipython-input-516823870.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
def run_epoch_train(model, loader):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for x, y in tqdm(loader, desc="Train", leave=False):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast():
            logits = model(x)
            loss = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * y.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    return total_loss / total, correct / total


@torch.no_grad()
def run_epoch_eval(model, loader):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    for x, y in tqdm(loader, desc="Eval", leave=False):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        with torch.cuda.amp.autocast():
            logits = model(x)
            loss = criterion(logits, y)

        total_loss += loss.item() * y.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    return total_loss / total, correct / total


In [ ]:
best_val_acc = -1.0
pat = 0
history = []

for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_acc = run_epoch_train(model, train_loader)
    va_loss, va_acc = run_epoch_eval(model, val_loader)

    history.append({
        "epoch": epoch,
        "train_loss": float(tr_loss),
        "train_acc": float(tr_acc),
        "val_loss": float(va_loss),
        "val_acc": float(va_acc),
    })

    print(f"Epoch {epoch}/{EPOCHS} | train acc {tr_acc:.4f} | val acc {va_acc:.4f}")

    if va_acc > best_val_acc:
        best_val_acc = va_acc
        pat = 0
        torch.save({
            "model_name": MODEL_NAME,
            "model_state": model.state_dict(),
            "label_map": label_map,
            "best_val_acc": float(best_val_acc)
        }, BEST_CKPT_PATH)
        print("✅ Saved best:", BEST_CKPT_PATH)
    else:
        pat += 1
        if pat >= PATIENCE:
            print("⏹️ Early stop")
            break

print("✅ Best val acc:", best_val_acc)


Train:   0%|          | 0/39 [00:00<?, ?it/s]/tmp/ipython-input-4214972945.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Eval:   0%|          | 0/9 [00:00<?, ?it/s]/tmp/ipython-input-4214972945.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 1/20 | train acc 0.3852 | val acc 0.3511
✅ Saved best: /content/drive/MyDrive/SkinCare_AI_Component/models/vision/skin_type_convnext_cleaned_best.pt


Epoch 2/20 | train acc 0.4085 | val acc 0.4074
✅ Saved best: /content/drive/MyDrive/SkinCare_AI_Component/models/vision/skin_type_convnext_cleaned_best.pt


Epoch 3/20 | train acc 0.4192 | val acc 0.4179
✅ Saved best: /content/drive/MyDrive/SkinCare_AI_Component/models/vision/skin_type_convnext_cleaned_best.pt


Epoch 4/20 | train acc 0.4437 | val acc 0.3721


...

Epoch 16/20 | train acc 0.8778 | val acc 0.7023


Epoch 17/20 | train acc 0.8935 | val acc 0.7233
✅ Saved best: /content/drive/MyDrive/SkinCare_AI_Component/models/vision/skin_type_convnext_cleaned_best.pt


Epoch 18/20 | train acc 0.9076 | val acc 0.7166


Epoch 19/20 | train acc 0.9260 | val acc 0.6641


Epoch 20/20 | train acc 0.9197 | val acc 0.7405
✅ Saved best: /content/drive/MyDrive/SkinCare_AI_Component/models/vision/skin_type_convnext_cleaned_best.pt
✅ Best val acc: 0.7404580152671756


In [ ]:
ckpt = torch.load(BEST_CKPT_PATH, map_location=device)
best_model = timm.create_model(ckpt["model_name"], pretrained=False, num_classes=num_classes).to(device)
best_model.load_state_dict(ckpt["model_state"], strict=True)
best_model.eval()

y_true, y_pred = [], []

with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Test"):
        x = x.to(device, non_blocking=True)
        logits = best_model(x)
        pred = logits.argmax(dim=1).cpu().numpy().tolist()
        y_true.extend(y.numpy().tolist())
        y_pred.extend(pred)

test_acc = accuracy_score(y_true, y_pred)
test_f1  = f1_score(y_true, y_pred, average="macro")

print("✅ Test accuracy:", test_acc)
print("✅ Test macro F1:", test_f1)

print("\nClassification report:\n")
print(classification_report(
    y_true, y_pred,
    target_names=[id_to_label[i] for i in range(num_classes)]
))


Test: 100%|██████████| 9/9 [00:06<00:00,  1.43it/s]


✅ Test accuracy: 0.7326355851569933
✅ Test macro F1: 0.7209254137161887

Classification report:

              precision    recall  f1-score   support

        oily       0.72      0.70      0.71       369
         dry       0.75      0.81      0.78       446
 combination       0.72      0.64      0.68       236

    accuracy                           0.73      1051
   macro avg       0.73      0.72      0.72      1051
weighted avg       0.73      0.73      0.73      1051



In [ ]:
cm = confusion_matrix(y_true, y_pred, labels=list(range(num_classes)))
cm_df = pd.DataFrame(
    cm,
    index=[f"true_{id_to_label[i]}" for i in range(num_classes)],
    columns=[f"pred_{id_to_label[i]}" for i in range(num_classes)]
)

metrics = {
    "model_name": MODEL_NAME,
    "best_val_acc": float(best_val_acc),
    "test_acc": float(test_acc),
    "test_macro_f1": float(test_f1),
    "label_map": label_map,
    "history": history
}

with open(METRICS_JSON, "w") as f:
    json.dump(metrics, f, indent=2)

cm_out = RESULTS_DIR / "convnext_cleaned_confusion_matrix.csv"
cm_df.to_csv(cm_out)

print("✅ Saved metrics:", METRICS_JSON)
print("✅ Saved confusion matrix:", cm_out)
cm_df


✅ Saved metrics: /content/drive/MyDrive/SkinCare_AI_Component/results/skin_type/convnext_cleaned_metrics.json
✅ Saved confusion matrix: /content/drive/MyDrive/SkinCare_AI_Component/results/skin_type/convnext_cleaned_confusion_matrix.csv


,pred_oily,pred_dry,pred_combination
true_oily,257,81,31
true_dry,56,363,27
true_combination,43,43,150
